In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from tcn import TCN
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import matplotlib.pyplot as plt

In [3]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
df = pd.read_csv('main.csv')

In [11]:
def evaluate_model(y_true, y_pred):
    mae = mean_absolute_error(y_true, y_pred)
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    mape = np.mean(np.abs((y_true - y_pred) / y_true)) * 100
    r2 = r2_score(y_true, y_pred)

    return {
        'MAE': mae,
        'MSE': mse,
        'RMSE': rmse,
        'MAPE (%)': mape,
        'R2 Score': r2
    }

In [12]:
def create_sequences(data, n_steps):
    X, y = [], []
    for i in range(n_steps, len(data)):
        X.append(data[i - n_steps:i])
        y.append(data[i])
    return np.array(X), np.array(y)

In [16]:
total_pred = []
total_test = []

In [17]:

# Подготовка данных для одного района
rayon_id = 126
for rayon_id in range(1, 127):
    df_rayon = df[df['Район'] == rayon_id].sort_values('Год')
    prices = df_rayon['Цена'].values.reshape(-1, 1)
    
    # Масштабирование
    scaler = MinMaxScaler()
    scaled_prices = scaler.fit_transform(prices)
    
    # Функция для создания последовательностей
    
    
    n_steps = 3
    X_all, y_all = create_sequences(scaled_prices, n_steps)
    
    # Train/test split (последние 2 года — тест)
    X_train, X_test = X_all[:-2], X_all[-2:]
    y_train, y_test = y_all[:-2], y_all[-2:]
    
    model = Sequential()
    model.add(TCN(input_shape=(n_steps, 1)))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mse')
    model.fit(X_train, y_train, epochs=200, verbose=0)
    
    y_pred = model.predict(X_test)
    y_pred_inv = scaler.inverse_transform(y_pred)
    y_test_inv = scaler.inverse_transform(y_test)

    total_test.append(y_test_inv)
    total_pred.append(y_pred_inv)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 305ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 294ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 314ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 342ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 312ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 324ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 342ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 292ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 359ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 334ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 322ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 299ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 329ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 307ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 347ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 332ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 313ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 330ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 326ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 314ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 296ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 327ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 320ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 333ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 326ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 275ms/step


In [20]:
evaluate_model(y_test_inv, y_pred_inv)

{'MAE': 4543.07553449113,
 'MSE': 20907010.303812504,
 'RMSE': np.float64(4572.418430525853),
 'MAPE (%)': np.float64(1.9297581927154261),
 'R2 Score': -16.07016918598337}

In [18]:
y_test_inv

array([[236659.81756483],
       [234446.43212085]])

In [19]:
y_pred_inv

array([[232633.92],
       [239506.69]], dtype=float32)

In [22]:
total_test = np.vstack(total_test).flatten()

In [23]:
total_pred = np.vstack(total_pred).flatten()

In [24]:
evaluate_model(total_test, total_pred)

{'MAE': 10728.487788896975,
 'MSE': 194664455.61080828,
 'RMSE': np.float64(13952.220454494269),
 'MAPE (%)': np.float64(3.7897770194606344),
 'R2 Score': 0.9605575885977304}